# Q-Learning

In [13]:
import numpy as np
import pandas as pd
import random

# TODO: DEFINE ACTION SPACE FOR EACH STATE

### Define States ###
states = []
for i in range(14):
    for j in range(20):
        states.append((i+1, j+1))

# Left wall states
left_wall = []
for i in range(16):
    left_wall.append((i, 0))
# Upper wall states
upper_wall = []
for j in range(22):
    upper_wall.append((15, j))
# Right wall
right_wall = []
for i in range(16):
    left_wall.append((i, 21))
# Bottom wall
bottom_wall = []
for j in range(22):
    upper_wall.append((0, j))

# Concat to walls
walls = left_wall + upper_wall + right_wall + bottom_wall

# Vertical wall + tree (right from Trainer)
for i in range(4, 14+1):
	walls.append((i, 4))

# Horizontal trees + building + mountain
for i in range(9, 16):
    for j in range(4, 20+1):
	    walls.append((i, j))
walls.remove((9,8)) # Do not add door to walls

# Horizontal lower wall 
for j in range(4, 16+1):
	walls.append((4, j))
walls.remove((4,8)) # Do not add staircase to walls

# Vertial Call right from charmander
for i in range(3, 9+1):
	walls.append((i, 19)) 
    
# Vertical Wall left from charmander
walls.append((8, 12))
walls.append((7, 12))
for j in range(12, 20):
    walls.append((6, j))
walls.remove((6, 16))


### Define Rewards ###
rewards = {}
for state in states:
    # Catch Bisasam
    #if state == (8, 10):
        #rewards[state] = 100
    # Catch Schiggy
    #elif state == (2, 13):
        #rewards[state] = 100
    # Catch Charmander
    if state == (8, 18):
        rewards[state] = 100
    # Go to building
    #elif state == (9, 8):
        #rewards[state] = 1
    # For all other States
    else:
        rewards[state] = -0.5
    
# Function that returns the valid next state, given current state and action
def getNextState(a: str, s: tuple) -> tuple:
    if a == "U":
        next_state = (s[0] - 1, s[1])
        if next_state in walls:
            next_state = (s[0], s[1])

    if a == "D":
        next_state = (s[0] + 1, s[1])
        if next_state in walls:
            next_state = (s[0], s[1])    

    if a == "L":
        next_state = (s[0], s[1] - 1)
        if next_state in walls:
            next_state = (s[0], s[1])

    if a == "R":
        next_state = (s[0], s[1] + 1)
        if next_state in walls:
            next_state = (s[0], s[1])
    
    return next_state


### Define Actions ###
actions = {}
for s in states:
    temp_action = []

    if getNextState("U", s) != s:
        temp_action.append("U")

    if getNextState("D", s) != s:
        temp_action.append("D")

    if getNextState("L", s) != s:
        temp_action.append("L")

    if getNextState("R", s) != s:
        temp_action.append("R")

    actions[s] = temp_action


################################################
### Define Q-Table and initialize it with 0  ###
################################################
n_states = len(states)      
n_action = 4

# Initialize Q_Table to all Zeros and convert to pd DataFrame
Q = np.zeros(shape=(n_states, n_action))
Q = pd.DataFrame(Q, columns=["U", "D", "L", "R"], index=states)

print("Q-Table at the beginning:")
display(Q)

### Define Hyperparams ###
GAMMA = 0.9
EPSILON = 0.95
ALPHA = 0.9
EPISODE = 1000

# Q-Learning algorithm:
for i in range(EPISODE):
    reward_history = []

    # Init (a, b) to (1, 4) so that the while loop always triggers and shuffles new
    (a, b) = (0, 0)

    # Get random start state
    while (a, b) in walls:
        a = np.random.choice(a=np.arange(15))
        b = np.random.choice(a=np.arange(21))

    state = (a, b)
    if i % 100 == 0: print(f"Start State at Epsiode {i} is {state}")
    
    # Simulate Episode
    for i in range(100):
        #print(f"Current state: {state}")
        
        # Get next state randomly (Exploration)
        # Epsilon decreases in relation to the current episode
        # Thus Epsilon is high at the beginning and low at the end
        if random.random() < EPSILON**i**(1/3):
            # Get random action
            a = np.random.choice([action for action in actions[state]])
            next_state = getNextState(a, state)
            #print(f"Random Action '{a}' has been taken. Resulting next state is {next_state}")

        # Get next state from Q-Table (Exploitation)
        else:
            valid_actions = [action for action in actions[state]]
            #print(f"Valid Action in state {state} is: {valid_actions}")
            # Get max_a q(state, a) from the Q-Table
            a = list(Q.loc[[state], [action for action in actions[state]]].idxmax(axis=1).values)[0]
            #print(f"Max Action in state {state} is: {a}")
            next_state = getNextState(a, state)
            #print(f"Next state is: {next_state}")

        # Update Q-Table
        Q.at[state, a] = Q.at[state, a] + ALPHA * (rewards[next_state] + GAMMA * float(Q.loc[[next_state], :].max(axis=1)) - Q.at[state, a])
        reward_history.append(rewards[next_state])
                
        # If we reach one terminal state: end the episode
        if next_state == (8, 18):
            break

        state = next_state
        

print("Training is finished!")
print("Q-Table in the end:")
display(Q)


Q-Table at the beginning:


,U,D,L,R
"(1, 1)",0.0,0.0,0.0,0.0
"(1, 2)",0.0,0.0,0.0,0.0
"(1, 3)",0.0,0.0,0.0,0.0
"(1, 4)",0.0,0.0,0.0,0.0
"(1, 5)",0.0,0.0,0.0,0.0
...,...,...,...,...
"(14, 16)",0.0,0.0,0.0,0.0
"(14, 17)",0.0,0.0,0.0,0.0
"(14, 18)",0.0,0.0,0.0,0.0
"(14, 19)",0.0,0.0,0.0,0.0


Start State at Epsiode 0 is (8, 14)
Start State at Epsiode 100 is (2, 11)
Start State at Epsiode 200 is (7, 6)
Start State at Epsiode 300 is (3, 5)
Start State at Epsiode 400 is (3, 14)
Start State at Epsiode 500 is (10, 3)
Start State at Epsiode 600 is (6, 3)
Start State at Epsiode 700 is (4, 1)
Start State at Epsiode 800 is (12, 3)
Start State at Epsiode 900 is (7, 3)
Training is finished!
Q-Table in the end:


,U,D,L,R
"(1, 1)",0.0,18.858038,0.000000,19.330718
"(1, 2)",0.0,22.694402,14.871736,22.542261
"(1, 3)",0.0,25.728486,19.776724,24.469180
"(1, 4)",0.0,29.112467,21.138395,23.004435
"(1, 5)",0.0,26.116039,20.203992,32.621942
...,...,...,...,...
"(14, 16)",0.0,0.000000,0.000000,0.000000
"(14, 17)",0.0,0.000000,0.000000,0.000000
"(14, 18)",0.0,0.000000,0.000000,0.000000
"(14, 19)",0.0,0.000000,0.000000,0.000000


# Print optimal Policy 🚀


In [15]:
def printBestActionForEachState():
    # Print optimal policy 
    for state in states:
        # Skip terminal states and wall
        if state in walls:
            pass
        # All other non-terminal states 
        else:
            best_action = list(Q.loc[[state], :].idxmax(axis=1).values)[0]
            print(f"The best action in state {state} is {best_action}")

rewards_hist = []
# Recursive function to draw path beginning at s (should be s = (14, 2))
def printPath(s):
    if s == (8, 18):
        print(s)
        print("Terminal state has been reached.")
        return 
    # Get best action from Q-Table and call function with next state
    print(s)
    best_action = str(list(Q.loc[[s], :].idxmax(axis=1).values)[0])
    next_state = getNextState(best_action, s)
    # Add reward of the transition into the record list
    rewards_hist.append(rewards[next_state])
    printPath(next_state)

printPath((14, 2))

print(f"Total Reward collected {sum(rewards_hist)}")


(14, 2)
(13, 2)
(12, 2)
(11, 2)
(10, 2)
(9, 2)
(8, 2)
(7, 2)
(6, 2)
(5, 2)
(4, 2)
(3, 2)
(3, 3)
(3, 4)
(3, 5)
(3, 6)
(3, 7)
(3, 8)
(4, 8)
(5, 8)
(5, 9)
(5, 10)
(5, 11)
(5, 12)
(5, 13)
(5, 14)
(5, 15)
(5, 16)
(6, 16)
(7, 16)
(8, 16)
(8, 17)
(8, 18)
Terminal state has been reached.
Total Reward collected 84.5
